# SAP TechEd 2025, session DA261
## Unlocking AI-driven insights from your business data in SAP HANA Cloud

## Exercise 0 - Getting started

### SAP Business Application Studio - Python environment setup and configuration 

- Install required Python modules
- Restart Python kernel
- Prepare the SAP HANA Cloud connection credential file
- Validate SAP HANA Cloud connectivity



### Installation of Python modules

#### Package overview
The following Python modules are to be installed during this hands-on introduction. 

__hdbcli__
The Python Database API Specification v2.0 (PEP 249) defines a set of methods that provides a consistent database interface independent of the actual database being used. The Python extension module for SAP HANA implements PEP 249. Once you install the module, you can access and change the information in SAP HANA databases from Python.  
You can find the documentation at https://help.sap.com/docs/SAP_HANA_CLIENT/f1b440ded6144a54ada97ff95dac7adf/f3b8fabf34324302b123297cdbe710f0.html  
You can find the package at https://pypi.org/project/hdbcli/  

__hana-ml__
The Python Machine Learning client for SAP HANA package enables Python data scientists to access SAP HANA data using SAP HANA dataframes and utilize SAP HANA's database AI engine algorithm libraries (PAL and APL) and NLP services to build and apply machine learning models within the database natively. While scripting in python, SQL is generated on the fly and executed in the SAP HANA database. The SAP HANA dataframe is simply a SQL select statement against any data in the SAP HANA database, data is not copied into the python environment, unless explicitly collected.  
You can find the documentation at: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/latest/en-US/hana_ml.html   
You can find the package at https://pypi.org/project/hana-ml/  


</br>

#### Install Python packages

Run the following package installations. 
- **pip** is the package installer for Python. 
- You can use pip to install packages from the Python Package Index and other indexes.

In [ ]:
!pip install hdbcli --break-system-packages
!pip install hana-ml --break-system-packages
#!pip install generative-ai-hub-sdk[all] --break-system-packages
!pip install folium --break-system-packages
!pip install ipywidgets --break-system-packages

!pip install plotly --break-system-packages
!pip install nbformat --break-system-packages

# kernel restart required!!!


#### Restart Python kernel ...

The Python kernel to be used with the Jupyter Notebook requires now a restart after package installation.

Once the libraries have been successfully installed, the kernel must be restarted. Do this by clicking on the Restart option in the menu bar at the top.
   <br>![](./images/BAS_python_env_restart.png)



> **Note** This may take a minute.

### Prepare the SAP HANA Cloud connection credential file

The Python script further below, get used to connect this Python environment to the SAP HANA Cloud instance.  
The script
- first imports database connection modules 
- then initiates a connection using the connection details provided in a local file (user.ini)
- the script is reused for creating the connection from python to SAP HANA Cloud in each exercise
- Alternatively, the user would be prompted to enter hostname, their username and password, which are then used to establish a secure connection to the SAP HANA Cloud database. 

</br>

**Note** Use your username and password supplied by the session instructor to logon to the SAP HANA Cloud database.   
Find the **host_address** in the lesson content  __temp_user.ini__ file.

</br>

👉🏻 **Before running the next cell** make sure:

1. The content of the [./ex0/temp_user.ini](ex0/temp_user.ini) file is copied to a file [./ex0/user.ini](ex0/user.ini) and 
2. the user and the password given to you by the instructor are updated in `user.ini` file 


👇🏼  
![Set user](./images/BAS_python_edit_userini.png)

### Connect to the HANA Cloud

Now executed the provided Python code cells below, by
- __click__ on the __PLAY-icon__ left to the cell
- or with the cell selected press __Ctrl+Alt+Enter__
- it is also possible to click into the code cell and press __Shift+Enter__

![Execute Python cell](./images/BAS_python_execute_cell.png)

</br>


<br>
This first python script cell, reads the ./user.ini-file and extracts the connection details into variables.

In [ ]:
import os, sys

file_path = './user.ini'  # Replace this with the path to your file

if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
    from hana_ml.algorithms.pal.utility import Settings
    myhost, myport, myuser, mypwd = Settings.load_config(file_path)
    if myhost == '' or myport == 0 or myuser == '' or mypwd == '':
        print("At least one required connection parameter is empty.")
    else:
        print(f"Connection to SAP HANA db at {myhost}:{myport} \nwill be attempted for the user {myuser}")
else:
    sys.exit(f"The file '{file_path}' does not exist! Create and configure it first.")
    

<br>

The variables with the connection details are now used to establish the SAP HANA Cloud database connection from Python.
- From the hana_ml dataframe class, the ConnectionContext subclass is used to create a connection with the Python client (hdbcli) for SAP HANA underneath.

In [ ]:
from hana_ml import dataframe as hdf
myconn=hdf.ConnectionContext(
    address=myhost, 
    port=myport, 
    user=myuser,
    password=mypwd
)
print(f"Connected to SAP HANA db version {myconn.hana_version()} \nat {myhost}:{myport} as user {myuser}")

<br>

The ConnectionContext object allows us to do various things, with the HANA dataframe sql()-method we quickly seek to issue a quick sql-query to the database to validate the connection.

In [ ]:
print(f'Current time on the SAP HANA server: {myconn.sql("SELECT NOW() FROM DUMMY").collect().CURRENT_TIMESTAMP[0]}')

<br>

In the following, alternative connection credential configuration methods are illustrated as reference information.  
__Those shall not be executed__

Using user input prompts for the connection credentials